In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
def cleanTweet(originalTweet):
    htmlParser = HTMLParser.HTMLParser()

    tweet = originalTweet
    #tweet = htmlParser.unescape(originalTweet);
    #tweet = tweet.decode('windows-1252').encode('ascii', 'ignore')
    #tweet = tweet.decode('windows-1252')
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #tweet = re.sub(r'[^\x00-\xFF]+', r'', tweet)
    #tweet = re.sub(r'[^\x00-\x7F]+', r'', tweet)
    #tweet = tweet.decode('utf-8').strip()
    #tweet = tweet.decode('unicode_escape').encode('ascii','ignore')
    #tweet = tweet.encode('ascii','ignore')
    tweet = ''.join([i if ord(i) < 128 else ' ' for i in tweet])
    
    # remove URLs in tweet
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)

    # remove HTML tags from tweet
    tweet = re.sub('<[^<]+?>', '', tweet)

    # remove strings starting with @ in tweet
    tweet = re.sub(r'(\s)@\w+', r'', tweet)

    # remove exclamations
    tweet = re.sub(r'!', r' ', tweet)

    # separates words joined with capital words.
    # E.g. DisplayIsAweson to Display Is Awesom
    #tweet = " ".join(re.findall('[A-Z][^A-Z]*', tweet));

    # remove extra white spaces
    tweet = re.sub(r'\s+', r' ', tweet)
    
    return tweet

In [3]:
#data = pd.read_csv("A:\\new_Sync\\Box Sync\\academics\\sem3\\583\\project\\TwitterAnalysis\\data\\trainRomney_3_columns.csv")

In [4]:
data = pd.read_csv("C:\\Users\\shekh\\Desktop\\twitteranalysis\\src\\trainObama_3_columns.csv")

In [5]:
len(data)

5624

In [6]:
rawTweetsSeries = data['Anootated tweet'];
tweetLabels = data['Class'];

In [7]:
rawTweetsList = rawTweetsSeries.tolist()
tweetLabelList = tweetLabels.tolist()

In [8]:
len(rawTweetsList)

5624

In [9]:
# clean all the tweets
i = 0;
cleanedTweetsList = []
for tweet in rawTweetsList:
    #tweet.encode('utf-8').strip()
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #print (i ,),
    cleanedTweet = cleanTweet(tweet).encode('ascii', 'ignore').strip();
    cleanedTweetsList.append(cleanedTweet);
    
    i += 1

In [10]:
cleanedTweetsList[12]

'@realDonaldTrump I agree we cant afford 4 more years of Obama'

In [11]:
print (i)
print (len(cleanedTweetsList))

5624
5624


In [12]:
#with open('cleanedRomney.csv', 'wb') as f:
#    writer = csv.writer(f)
#    writer.writerows(izip(cleanedTweetsList, tweetLabelList))

In [13]:
trainTweets, testTweets, trainTweetLabels, testTweetLabels = train_test_split(cleanedTweetsList, tweetLabelList, 
                                                                              test_size=0.25, random_state = 40);

In [14]:
len(trainTweetLabels)

4218

In [15]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                             max_df = 0.75,
                             sublinear_tf=True,
                             use_idf=True)

In [16]:
train_vectors = vectorizer.fit_transform(trainTweets);
test_vectors = vectorizer.transform(testTweets);
total_vectors = vectorizer.fit_transform(cleanedTweetsList);

In [17]:
type(train_vectors)

scipy.sparse.csr.csr_matrix

In [18]:
#classifier_rbf = svm.SVC()      default kernel RBF. Very poor results, classify all tweets as -1
#classifier_rbf = svm.SVC(kernel='linear')     decent results, ~50-60 accuracy
#classifier_rbf = svm.LinearSVC()
knn = KNeighborsClassifier()

In [19]:
knn.fit(train_vectors, trainTweetLabels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [20]:
prediction_rbf = knn.predict(test_vectors)

In [21]:
predictedLabelList = prediction_rbf.tolist()

In [22]:
classActual = np.array(testTweetLabels)

classPredicted = np.array(predictedLabelList)

In [23]:
classActual
len(classActual)

classPredicted
len(classPredicted)

1406

In [24]:
target_names = ['-1', '0', '1']

In [25]:
prfs(classActual, classPredicted)

(array([ 0.53398058,  0.39543058,  0.36239782]),
 array([ 0.11578947,  0.45638945,  0.60730594]),
 array([ 0.19031142,  0.42372881,  0.45392491]),
 array([475, 493, 438]))

In [26]:
print (len(predictedLabelList))
print (len(testTweetLabels))

1406
1406


In [27]:
correct = 0;
for i in range(len(predictedLabelList)):
    if(predictedLabelList[i] == testTweetLabels[i]):
        correct += 1;

In [28]:
print (correct)

546


In [29]:
predictedLabelList[470]

-1L

In [30]:
import collections
counter=collections.Counter(predictedLabelList)
print(counter)

Counter({1L: 734, 0L: 569, -1L: 103})


In [31]:
import collections
counter=collections.Counter(testTweetLabels)
print(counter)

Counter({0: 493, -1: 475, 1: 438})


In [32]:
knn.score(test_vectors, testTweetLabels)

0.38833570412517782

In [33]:
print(classification_report(testTweetLabels, predictedLabelList, target_names=target_names))

             precision    recall  f1-score   support

         -1       0.53      0.12      0.19       475
          0       0.40      0.46      0.42       493
          1       0.36      0.61      0.45       438

avg / total       0.43      0.39      0.35      1406



In [34]:
cvScores = cross_val_score(knn, total_vectors, tweetLabelList, cv=10,scoring='f1_macro')

In [35]:
print ("Cross validations scores: "),
print (cvScores)

Cross validations scores:  [ 0.37047013  0.33532492  0.41618448  0.40096183  0.36519739  0.38130038
  0.43932854  0.35902998  0.3630798   0.3505088 ]


In [36]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.37813862405912596)
('Minimum: ', 0.33532491767764361)
('Maximum: ', 0.43932853802758487)


In [37]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens